In [1]:
%load_ext autoreload

In [2]:
from options.options import Options
import os
import torch
from build_dataset_model import build_loaders, build_model
from utils import get_model_attr, calculate_model_losses, tensor_aug
from collections import defaultdict
import math

from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm
import numpy as np

from new.CustomVAE import *
from utils import calculate_model_losses

In [3]:
args = Options().parse()
if (args.output_dir is not None) and (not os.path.isdir(args.output_dir)):
    os.mkdir(args.output_dir)
if (args.test_dir is not None) and (not os.path.isdir(args.test_dir)):
    os.mkdir(args.test_dir)

# no KL divergence loss
args.use_AE = True

# tensorboard
# writer = SummaryWriter()

# load data
vocab, train_loader, val_loader = build_loaders(args)

dt = train_loader.dataset

| options
dataset: suncg
suncg_train_dir: metadata/data_rot_train.json
suncg_val_dir: metadata/data_rot_val.json
suncg_data_dir: /home/yizhou/Research/SUNCG/suncg_data
loader_num_workers: 8
embedding_dim: 64
gconv_mode: feedforward
gconv_dim: 128
gconv_hidden_dim: 512
gconv_num_layers: 5
mlp_normalization: batch
vec_noise_dim: 0
layout_noise_dim: 32
batch_size: 16
num_iterations: 20000
eval_mode_after: -1
learning_rate: 0.0001
print_every: 100
checkpoint_every: 1000
snapshot_every: 10000
output_dir: ./checkpoints
checkpoint_name: latest_checkpoint
timing: False
multigpu: False
restore_from_checkpoint: False
checkpoint_start_from: None
test_dir: ./layouts_out
gpu_id: 0
KL_loss_weight: 0.1
use_AE: False
decoder_cat: True
train_3d: True
KL_linear_decay: False
use_attr_30: True
manual_seed: 42
batch_gen: False
measure_acc_l1_std: False
heat_map: False
draw_2d: False
draw_3d: False
fine_tune: False
gan_shade: False
blender_path: /home/yizhou/blender-2.92.0-linux64/blender

Starting to read 

In [4]:
args.use_AE = False

In [5]:
for batch in tqdm(train_loader):
    #t += 1
    ids, objs, boxes, triples, angles, attributes, obj_to_img, triple_to_img = tensor_aug(batch)
    z = torch.randn(objs.size(0), 64).to(objs.device)
    break

  0%|          | 0/3367 [00:00<?, ?it/s]

In [6]:
obj_to_img.shape

torch.Size([226])

In [7]:
from new.CustomVAE import TransformerEncoder

In [8]:
kwargs = {
        'vocab': dt.vocab,
        'batch_size': args.batch_size,
        'train_3d': args.train_3d,
        'decoder_cat': args.decoder_cat,
        'embedding_dim': 84,
        'gconv_mode': args.gconv_mode,
        'gconv_num_layers': args.gconv_num_layers,
        'mlp_normalization': args.mlp_normalization,
        'vec_noise_dim': args.vec_noise_dim,
        'layout_noise_dim': args.layout_noise_dim,
        'use_AE': args.use_AE
    }

In [9]:
model = TransformerEncoder(**kwargs)

In [10]:
model = model.cuda()

In [11]:
score_matrix = model.encoder(objs, boxes, angles, attributes, obj_to_img)

In [12]:
all_samples, all_log_probs = model.sample(score_matrix, obj_to_img)

In [13]:
log_prob = torch.sum(all_log_probs)

In [14]:
log_prob.backward()

In [15]:
dt.vocab

{'object_idx_to_name': ['__room__',
  'curtain',
  'shower_curtain',
  'dresser',
  'counter',
  'bookshelf',
  'picture',
  'mirror',
  'floor_mat',
  'chair',
  'sink',
  'desk',
  'table',
  'lamp',
  'door',
  'clothes',
  'person',
  'toilet',
  'cabinet',
  'floor',
  'window',
  'blinds',
  'wall',
  'pillow',
  'whiteboard',
  'bathtub',
  'television',
  'night_stand',
  'sofa',
  'refridgerator',
  'bed',
  'shelves'],
 'object_name_to_idx': {'__room__': 0,
  'curtain': 1,
  'shower_curtain': 2,
  'dresser': 3,
  'counter': 4,
  'bookshelf': 5,
  'picture': 6,
  'mirror': 7,
  'floor_mat': 8,
  'chair': 9,
  'sink': 10,
  'desk': 11,
  'table': 12,
  'lamp': 13,
  'door': 14,
  'clothes': 15,
  'person': 16,
  'toilet': 17,
  'cabinet': 18,
  'floor': 19,
  'window': 20,
  'blinds': 21,
  'wall': 22,
  'pillow': 23,
  'whiteboard': 24,
  'bathtub': 25,
  'television': 26,
  'night_stand': 27,
  'sofa': 28,
  'refridgerator': 29,
  'bed': 30,
  'shelves': 31},
 'pred_idx_to_na

In [16]:
from utils import *

In [17]:
all_samples.shape

torch.Size([226])

In [18]:
objs

tensor([30, 27,  8, 27, 18, 20,  3, 18, 26,  1,  0, 30,  1, 20, 14,  0, 28, 16,
         8, 13, 30, 13, 16, 26, 16,  9, 26,  3, 13,  0, 14, 18, 27, 20, 27, 13,
        13, 30, 28, 18,  0,  1, 13, 20, 26, 18, 30, 13, 30, 14,  0, 11, 14, 14,
        30, 14, 18,  0, 13, 13, 13, 18, 27, 13, 11, 30,  3,  9, 18, 20,  1, 27,
         0, 18, 13, 20,  7,  1, 27, 11, 13, 27, 20, 30, 26, 18,  8, 20, 20,  3,
         1, 13,  6, 13, 18,  0, 19, 26, 30,  9, 26,  9, 11,  0, 20, 18, 30, 30,
         0,  6,  8, 14, 14, 20,  6,  1, 30,  6, 27,  0, 13,  6, 21, 11,  9, 13,
        30, 20, 21, 27,  8, 13, 21, 18,  6, 27,  6,  0,  9,  6, 18, 27,  8,  4,
        28,  9,  4,  4,  9,  9,  4, 26,  4,  4,  1, 20,  6,  9, 13, 12, 13, 18,
        26, 13,  4,  6, 13, 13, 14, 18, 31, 27, 11, 18, 30,  6, 31,  0, 30, 20,
        13, 20, 20, 13, 13, 20, 20, 11,  3, 27, 13, 20, 20, 27, 13, 11, 13, 13,
         8,  6, 26, 20, 13,  0, 20,  3,  6, 30, 20, 18, 18, 18, 14, 26, 18, 14,
         0, 26, 13,  9, 30, 18, 11, 14, 

In [19]:
boxes.shape

torch.Size([226, 6])

In [20]:
triples.shape

torch.Size([235, 3])

In [21]:
from new.utils import obtain_sampled_relations

In [23]:
boxes_numpy = boxes.cpu().data.numpy()

In [25]:
new_ts = obtain_sampled_relations(objs, all_samples, boxes_numpy, dt.vocab)

In [27]:
new_ts.shape

torch.Size([226, 3])